# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")


To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921684449125                   -0.69    5.9         
  2   -7.926164023415       -2.35       -1.22    1.0    223ms
  3   -7.926836768212       -3.17       -2.37    1.9    286ms
  4   -7.926861515599       -4.61       -3.03    2.6    252ms
  5   -7.926861642652       -6.90       -3.39    1.9    220ms
  6   -7.926861669844       -7.57       -3.79    1.9    236ms
  7   -7.926861680401       -7.98       -4.32    1.2    230ms
  8   -7.926861681759       -8.87       -4.84    2.0    225ms
  9   -7.926861681859      -10.00       -5.27    1.9    221ms
 10   -7.926861681871      -10.90       -5.89    1.9    237ms
 11   -7.926861681873      -11.95       -7.07    2.1    239ms
 12   -7.926861681873      -13.59       -7.45    3.4    276ms
 13   -7.926861681873      -14.75       -8.32    1.6    215ms


If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
# which directly yields an AbstractSystem.
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921654020320                   -0.69    5.8         
  2   -7.926161641675       -2.35       -1.22    1.0    218ms
  3   -7.926837960888       -3.17       -2.37    1.8    239ms
  4   -7.926861482966       -4.63       -3.06    2.6    287ms
  5   -7.926861649538       -6.78       -3.45    2.1    228ms
  6   -7.926861671943       -7.65       -3.85    2.0    224ms
  7   -7.926861680134       -8.09       -4.27    1.1    206ms
  8   -7.926861681677       -8.81       -4.74    1.6    242ms
  9   -7.926861681861       -9.74       -5.32    2.1    233ms
 10   -7.926861681870      -11.03       -5.70    1.9    228ms
 11   -7.926861681873      -11.65       -7.03    1.4    225ms
 12   -7.926861681873      -13.11       -7.56    3.2    278ms
 13   -7.926861681873   +  -14.57       -8.21    2.2    229ms


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [5]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921672680694                   -0.69    5.8         
  2   -7.926168404241       -2.35       -1.22    1.0    250ms
  3   -7.926839921527       -3.17       -2.37    1.8    229ms
  4   -7.926864935118       -4.60       -3.03    2.8    253ms
  5   -7.926865054075       -6.92       -3.40    1.9    209ms
  6   -7.926865080040       -7.59       -3.78    1.9    215ms
  7   -7.926865091474       -7.94       -4.33    1.2    218ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [6]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [7]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")
